<a href="https://colab.research.google.com/github/piachouaifaty/CSC375_OperatingSystems/blob/master/GA_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-Up

### Imports

In [154]:
import numpy as np
import gc
import seaborn as sns
import tensorflow as tf
from itertools import cycle
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.core import Dropout, Activation
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, \
     Dense, Input, Activation, Dropout, GlobalAveragePooling2D, AveragePooling2D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.utils.class_weight import compute_class_weight
import cv2
from cv2 import imread, resize # manipulating the images
from tensorflow.keras.optimizers import Adam
import os
import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
import itertools
from tensorflow.keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from datetime import datetime

import glob
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import applications
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.optimizers import RMSprop
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
#!pip install fast_ml
#from fast_ml.model_development import train_valid_test_split
import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_wine

import statistics as st
import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from re import M

#!pip3 install pygad
import pygad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 KB 3.6 MB/s eta 0:00:00


### Drive mounting

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Path setup

In [3]:
results_skin_dir = os.path.join('..', '/content/drive/MyDrive/Kaggle/skin_results')
base_skin_dir = os.path.join('..', '/content/drive/MyDrive/Kaggle/skin')

results_skin_dir = os.path.join('..', '/content/drive/MyDrive/Kaggle/skin_results')
base_skin_dir = os.path.join('..', '/content/drive/MyDrive/Kaggle/skin')

### Metadata

In [4]:
df_skin = pd.read_csv(os.path.join(base_skin_dir, 'HAM10000_metadata.csv'))

In [5]:
all_labels = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']
print('All Labels ({}): {}'.format(len(all_labels), all_labels))

All Labels (7): ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']


In [6]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

lesion_ID_dict = {
    'nv': 0,
    'mel': 1,
    'bkl': 2,
    'bcc': 3,
    'akiec': 4,
    'vasc': 5,
    'df': 6
}

# Lesion and it's abbriv.
lesion_names = ['Melanocytic nevi','Melanoma','Benign keratosis-like lesions ',
               'Basal cell carcinoma','Actinic keratoses','Vascular lesions',
               'Dermatofibroma']
lesion_names_short = ['nv','mel','bkl','bcc','akiec','vasc','df']

# Maping the lesion type and ID to a dict.
df_skin['lesion_type']=df_skin['dx'].map(lesion_type_dict)
df_skin['lesion_ID'] = df_skin['dx'].map(lesion_ID_dict)

### Train/Test/Val Split

In [7]:
x = np.load(os.path.join(base_skin_dir,'x.csv.npy'))
y = np.load(os.path.join(base_skin_dir,'y.csv.npy'))

x = np.array(x)
y = np.array(y)

# convert y (targets) array as required by softmax activation function
y_t = to_categorical(y, num_classes = 7)

print('dataset shape', x.shape)

dataset shape (26565, 100, 100, 3)


In [8]:
#train-test-val
X_train, X_rem, y_train, y_rem = train_test_split(x, 
                                                  y_t, 
                                                  train_size=0.7, 
                                                  random_state=50)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, 
                                                    y_rem, 
                                                    test_size=0.33, 
                                                    random_state=50) # 0.5 x 0.2 = 0.1

print('x Train dataset shape', X_train.shape)
print('x Val dataset shape', X_valid.shape)
print('x test dataset shape', X_test.shape)

x Train dataset shape (18595, 100, 100, 3)
x Val dataset shape (5339, 100, 100, 3)
x test dataset shape (2631, 100, 100, 3)


In [9]:
#Blending holdout set
X_valid1, X_holdout, y_valid1, y_holdout = train_test_split(X_valid, 
                                                    y_valid, 
                                                    test_size=0.5, 
                                                    random_state=20)

print("X_train: ", X_train.shape)
print("X validation: ", X_valid.shape)
print("X test: ", X_test.shape)
print("-------------")
print("X validation2: ", X_valid1.shape)
print("X holdout: ",X_holdout.shape)

X_train:  (18595, 100, 100, 3)
X validation:  (5339, 100, 100, 3)
X test:  (2631, 100, 100, 3)
-------------
X validation2:  (2669, 100, 100, 3)
X holdout:  (2670, 100, 100, 3)


## Trained models

### Helper functions

In [10]:
df_models = pd.read_csv(os.path.join(results_skin_dir, 'modelfilenames.csv'))
df_models
#this file contains all the trained model filenames

,Model,timestamp,full,bagging,blending
0,Model 1 - Custom CNN,1676796445,1676796445_model.h5,1676796445_bagging_model.h5,1676796445_blending_model.h5
1,Model 2 - Custom CNN,1676816510,1676816510_model.h5,1676816510_bagging_model.h5,1676816510_blending_model.h5
2,Model 3 - Xception,1676839766,1676839766_model.h5,1676839766_bagging_model.h5,1676839766_blending_model.h5
3,Model 4 - Custom CNN,1676842274,1676842274_model.h5,1676842274_bagging_model.h5,1676842274_blending_model.h5
4,Model 5 - Xception,1676913088,1676913088_model.h5,1676913088_bagging_model.h5,1676913088_blending_model.h5
5,Model 6 - Resnet,1676936146,1676936146_model.h5,1676936146_bagging_model.h5,1676936146_blending_model.h5
6,Model 7 - DenseNet201,1676987763,1676987763_model.h5,1676987763_bagging_model.h5,1676987763_blending_model.h5
7,Model 8 - InceptionV3,1676997761,1676997761_model.h5,1676997761_bagging_model.h5,1676997761_blending_model.h5
8,Model 9 - VGG19,1677017458,1677017458_model.h5,1677017458_bagging_model.h5,1677017458_blending_model.h5
9,Model 10 - InceptionResnet,1677026677,1677026677_model.h5,1677026677_bagging_model.h5,1677026677_blending_model.h5


In [11]:
df_models['full'][0]

'1676796445_model.h5'

In [12]:
#helper function load our models
def load_trained_model(modeltype, modelnumber):
  path = os.path.join(results_skin_dir, df_models[modeltype][modelnumber-1])
  print(path)  
  m = load_model(path)
  return m

### Storing predictions (DO NOT RE-RUN)

In [ ]:
#the models will be inputted into the GA as follows:
#list: [model_name, predictions_test, accuracy_full, bagging_predictions_test, bagging_accuracy, blending_predictions_holdout, bagging_accuracy]
#model[0]: model name

In [13]:
def generate_model(modelnumber, X_test, y_test, X_holdout, y_holdout):
  pre = False
  if modelnumber == 3:
    pre = True
    print("Model 3, now generating preprocessing functions")
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    test_datagen.fit(X_test)
    test_generator = test_datagen.flow(X_test, y_test)
    holdout_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    holdout_datagen.fit(X_holdout)
    holdout_generator = holdout_datagen.flow(X_holdout, y_holdout)

  model_types = ["full", "bagging", "blending"]
  mod_list = []
  mod_list.append("model"+str(modelnumber))

  for modtype in model_types:
    print("Now loading model", modelnumber, modtype)
    model = load_trained_model(modtype, modelnumber)
    if pre:
      if modtype=="blending":
        print("Now predicting on holdout set with preprocessing")
        y_pred = model.predict(preprocess_input(X_holdout))
        print("Now evaluating on test set")
        scores = model.evaluate(test_generator, verbose = 1)
        accuracy = scores[1]
      else:
        print("Now predicting on test set with preprocessing")
        y_pred = model.predict(preprocess_input(X_test))
        print("Now evaluating on test set")
        scores = model.evaluate(test_generator, verbose = 1)
        accuracy = scores[1]
      print(y_pred)
      print("Model", modelnumber, modtype, "accuracy:", accuracy)
      mod_list.append(y_pred)
      mod_list.append(accuracy)
      print("--------------------------------")
    else:
      if modtype=="blending":
        print("Now predicting on holdout set")
        y_pred = model.predict(X_holdout)
      else:
        print("Now predicting on test set")
        y_pred = model.predict(X_test)
      #general model accuracy is always evaluated on the test set
      #accuracies needed for weighted voting
      print("Now evaluating on test set")
      scores = model.evaluate(X_test, y_test, verbose = 1)
      accuracy = scores[1]
      mod_list.append(y_pred)
      mod_list.append(accuracy)
      print(y_pred)
      print("Model", modelnumber, modtype, "accuracy: ", accuracy)
      print("--------------------------------")
  return mod_list

#PREDICTION for models that require preprocessing
#model 3
#y_pred = model.predict(preprocess_input(X_test))

#EVALUATION for models that require preprocessing

#test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
#test_datagen.fit(X_test)
#test_generator = test_datagen.flow(X_test, y_test)
#scores = model.evaluate(test_generator, verbose = 1)
#print("Accuracy: %.2f%%" % (scores[1] * 100))

In [ ]:
#m1 = generate_model(1, X_test, y_test, X_holdout, y_holdout)
#m2 = generate_model(2, X_test, y_test, X_holdout, y_holdout)
#m3 = generate_model(3, X_test, y_test, X_holdout, y_holdout)
#m4 = generate_model(4, X_test, y_test, X_holdout, y_holdout)
#m5 = generate_model(5, X_test, y_test, X_holdout, y_holdout)
#m6 = generate_model(6, X_test, y_test, X_holdout, y_holdout)
#m7 = generate_model(7, X_test, y_test, X_holdout, y_holdout)
#m8 = generate_model(8, X_test, y_test, X_holdout, y_holdout)
#m9 = generate_model(9, X_test, y_test, X_holdout, y_holdout)

#BAGGING MODEL NEEDS RETRAINING


Now loading model 1 full
/content/drive/MyDrive/Kaggle/skin_results/1676796445_model.h5
Now predicting on test set
83/83 [==============================] - 8s 5ms/step
Now evaluating on test set
83/83 [==============================] - 1s 5ms/step - loss: 0.3989 - accuracy: 0.9160
[[1.0000000e+00 2.6464586e-12 1.2870426e-13 ... 4.2029156e-17
  1.0325957e-19 5.7178761e-18]
 [3.2583785e-01 6.7386419e-01 4.5232642e-05 ... 2.5078861e-04
  1.5990624e-06 4.6272916e-08]
 [1.5828029e-04 5.2317153e-03 9.9460381e-01 ... 1.7378407e-07
  5.2395812e-06 4.8535833e-09]
 ...
 [9.9935549e-01 1.3765309e-04 5.0599076e-04 ... 6.8605533e-07
  7.5394766e-08 3.5044508e-08]
 [2.9784633e-08 3.0320324e-09 7.4011384e-09 ... 1.0000000e+00
  2.2682413e-21 5.4195323e-16]
 [3.3333118e-04 9.9939966e-01 2.5037705e-05 ... 2.4186127e-04
  6.4305622e-10 5.2439022e-09]]
Model 1 full accuracy:  0.9160014986991882
--------------------------------
Now loading model 1 bagging
/content/drive/MyDrive/Kaggle/skin_results/1676796

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


83/83 [==============================] - 3s 29ms/step - loss: 1.2611 - accuracy: 0.6393
[[9.95833755e-01 1.00981978e-07 1.02040094e-05 ... 5.18495871e-11
  7.97825123e-06 4.14795196e-03]
 [5.05379438e-01 4.72618520e-01 9.32080671e-03 ... 1.37407606e-05
  2.01628310e-07 5.93459226e-10]
 [2.40880325e-02 3.81933153e-01 5.93960941e-01 ... 3.53216723e-09
  1.40633165e-05 5.06548581e-09]
 ...
 [9.60462987e-01 4.06610919e-03 3.54469195e-02 ... 1.22167867e-05
  1.48180064e-07 8.76685657e-09]
 [1.02935908e-02 5.82752079e-02 5.13098836e-01 ... 1.07224591e-01
  1.25103761e-04 9.18470207e-04]
 [2.81041879e-02 9.70836163e-01 2.39109053e-04 ... 6.41699065e-04
  3.58882840e-10 4.12901970e-11]]
Model 3 full accuracy: 0.6393006443977356
--------------------------------
Now loading model 3 bagging
/content/drive/MyDrive/Kaggle/skin_results/1676839766_bagging_model.h5
Now predicting on test set with preprocessing
83/83 [==============================] - 3s 27ms/step
Now evaluating on test set
83/83 [====

OSError: ignored

In [14]:
#m10 = generate_model(10, X_test, y_test, X_holdout, y_holdout)

Now loading model 10 full
/content/drive/MyDrive/Kaggle/skin_results/1677026677_model.h5
Now predicting on test set
83/83 [==============================] - 61s 689ms/step
Now evaluating on test set
83/83 [==============================] - 60s 671ms/step - loss: 2.9518 - accuracy: 0.8590
[[9.9999994e-01 3.5996778e-25 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [6.3889064e-03 9.9326742e-01 2.3055085e-05 ... 3.0018430e-04
  8.9424272e-09 1.1592128e-05]
 [1.1705299e-10 1.6105911e-02 9.7966611e-01 ... 3.8399408e-09
  9.5133394e-09 6.0905694e-09]
 ...
 [9.9949574e-01 4.8734414e-04 1.6864671e-05 ... 1.2552351e-11
  1.0409595e-08 5.5713225e-13]
 [8.2526534e-11 5.1807714e-09 2.7891609e-09 ... 1.0000000e+00
  1.9577994e-16 1.3016260e-12]
 [6.0145510e-04 9.9937648e-01 4.3381833e-06 ... 1.7489032e-05
  4.6645864e-11 2.0614729e-08]]
Model 10 full accuracy:  0.8589890003204346
--------------------------------
Now loading model 10 bagging
/content/drive/MyDrive/Kaggle/skin_result

In [15]:
#np.save(os.path.join(results_skin_dir,'model1'), m1)
#np.save(os.path.join(results_skin_dir,'model2'), m2)
#np.save(os.path.join(results_skin_dir,'model3'), m3)
#np.save(os.path.join(results_skin_dir,'model4'), m4)
#np.save(os.path.join(results_skin_dir,'model5'), m5)
#np.save(os.path.join(results_skin_dir,'model6'), m6)
#np.save(os.path.join(results_skin_dir,'model7'), m7)
#np.save(os.path.join(results_skin_dir,'model8'), m8)
#np.save(os.path.join(results_skin_dir,'model9'), m9)
#np.save(os.path.join(results_skin_dir,'model10'), m10)

/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


### Loading lightweight stored model predictions

In [13]:
model1 = np.load(os.path.join(results_skin_dir,'model1.npy'), allow_pickle=True)
model1 = list(model1)

model2 = np.load(os.path.join(results_skin_dir,'model2.npy'),allow_pickle=True)
model2 = list(model2)

model3 = np.load(os.path.join(results_skin_dir,'model3.npy'),allow_pickle=True)
model3 = list(model3)

model4 = np.load(os.path.join(results_skin_dir,'model4.npy'),allow_pickle=True)
model4 = list(model4)

model5 = np.load(os.path.join(results_skin_dir,'model5.npy'),allow_pickle=True)
model5 = list(model5)

model6 = np.load(os.path.join(results_skin_dir,'model6.npy'),allow_pickle=True)
model6 = list(model6)

model7 = np.load(os.path.join(results_skin_dir,'model7.npy'),allow_pickle=True)
model7 = list(model7)

model8 = np.load(os.path.join(results_skin_dir,'model8.npy'),allow_pickle=True)
model8 = list(model8)

model9 = np.load(os.path.join(results_skin_dir,'model9.npy'),allow_pickle=True)
model9 = list(model9)

model10 = np.load(os.path.join(results_skin_dir,'model10.npy'),allow_pickle=True)
model10 = list(model10)

### Model helper functions

In [14]:
#given class probabilities as y_pred and real labels as y_test
def test_model(y_pred, y_test):
    """ Tunning the accurate results and inaccurate results

    Returns:
        (total, accurate) [tuple]: tuple of total tested test-cases, accurate
    """
    total = 0
    accurate = 0
    accurateindex = []
    wrongindex = []
    for i in range(len(y_pred)):
        if np.argmax(y_pred[i]) == np.argmax(y_test[i]):
            accurate += 1
            accurateindex.append(i)
        else:
            wrongindex.append(i)
        total += 1
    return (total, accurate)

In [15]:
#function that returns a 3D array where the first dimension is the model, and the other 2 are the predictions for each of the 7 classes
#this will be used for both voting and generating the training data for the metalearner through holdout set predictions
#where models is the list of models as generated by generate_models
#and modeltype is either 

#the models will be inputted into the GA as follows:
#list: [model_name, predictions_test, accuracy_full, bagging_predictions_test, bagging_accuracy, blending_predictions_holdout, bagging_accuracy]
#model[0]: model name

#the prediction index in the model generated by generate_model
def get_pred_idx(modtype):
    if modtype == "full":
        return 1
    elif modtype == "bagging":
        return 3
    elif modtype == "blending":
        return 5

#the accuracy index in the model generated by generate_model
def get_acc_idx(modtype):
    if modtype == "full":
        return 2
    elif modtype == "bagging":
        return 4
    elif modtype == "blending":
        return 6

def get_accuracies(models, model_type):
  num_models = len(models)
  test_accuracies = []
  idx = get_acc_idx(model_type)
  for m in models:
    test_accuracies.append(m[idx])
  return test_accuracies

def get_normalized_accuracies(models, model_type):
  accs = get_accuracies(models, model_type)
  normalized_accs=[float(i)/sum(accs) for i in accs]
  return normalized_accs

#models is a list of generate_model lists
def join_model_predictions(models, y_test, y_holdout, model_type):
  num_models = len(models)
  num_classes = y_test.shape[1]
  if model_type=="full" or model_type=="bagging":
    num_instances = y_test.shape[0]
  if model_type=="blending":  
    num_instances = y_holdout.shape[0]
    
  joined_preds = np.zeros(shape=(num_models, num_instances, num_classes))
  idx = int(get_pred_idx(model_type))
  count=0
  for i in models:
    #print(i[idx])
    joined_preds[count]=i[idx]
    count=count+1  
  return joined_preds

In [18]:
#models is a list of generate_model lists
#joins the predictions as crisp class labels rather than predictions

def join_model_predictions_crisp(models, y_test, y_holdout, model_type):
  num_models = len(models)
  num_classes = y_test.shape[1]
  if model_type=="full" or model_type=="bagging":
    num_instances = y_test.shape[0]
  if model_type=="blending":  
    num_instances = y_holdout.shape[0]
    
  joined_preds = np.zeros(shape=(num_models, num_instances, num_classes))
  idx = int(get_pred_idx(model_type))
  count=0
  for i in models:
    y_preds=i[idx]
    #print(y_preds)
    modelpreds=np.zeros_like(y_preds)
    modelpreds[np.arange(len(y_preds)), y_preds.argmax(1)] = 1
    #print(modelpreds)
    joined_preds[count]=modelpreds
    count=count+1  
  return joined_preds

In [21]:
#given a list of model lists (defined with the generate_model function), gets the soft voting accuracy
#for full and bagging
#mtype = "full" or "bagging"
def soft_voting(models, mtype, y_test, y_holdout):
    if mtype=="full":
      preds = join_model_predictions(models, y_test, y_holdout, "full")
    if mtype=="bagging":
      preds = join_model_predictions(models, y_test, y_holdout, "bagging")
    y_pred = np.mean(np.array(preds), axis=0)
    results = test_model(y_pred, y_test)
    accuracy = results[1]/results[0]
    print('Soft voting accuracy: ', accuracy)
    return accuracy

def weighted_soft_voting(models, mtype, y_test, y_holdout):
    if mtype=="full":
      preds = join_model_predictions(models, y_test, y_holdout, "full")
    if mtype=="bagging":
      preds = join_model_predictions(models, y_test, y_holdout, "bagging")
    accuracies = get_accuracies(models, mtype)
    y_pred = np.average(np.array(preds), axis=0, weights=accuracies)
    results = test_model(y_pred, y_test)
    accuracy = results[1]/results[0]
    print('Weighted soft voting accuracy: ', accuracy)
    return accuracy

In [22]:
#given a list of model lists (defined with the generate_model function), gets the hard voting accuracy
#for full and bagging
#mtype = "full" or "bagging"

def hard_voting(models, mtype, y_test, y_holdout):
    if mtype=="full":
      preds = join_model_predictions_crisp(models, y_test, y_holdout, "full")
    if mtype=="bagging":
      preds = join_model_predictions_crisp(models, y_test, y_holdout, "bagging")
    y_pred = np.sum(np.array(preds), axis=0)
    results = test_model(y_pred, y_test)
    accuracy = results[1]/results[0]
    print('Hard voting accuracy: ', accuracy)
    return accuracy

def weighted_hard_voting(models, mtype, y_test, y_holdout):
    if mtype=="full":
      preds = join_model_predictions_crisp(models, y_test, y_holdout, "full")
    if mtype=="bagging":
      preds = join_model_predictions_crisp(models, y_test, y_holdout, "bagging")
    accs = get_normalized_accuracies(models, mtype)
    weights = np.array(accs)
    weighted_preds = preds*weights[:,np.newaxis, np.newaxis]
    y_pred = np.sum(np.array(weighted_preds), axis=0)
    results = test_model(y_pred, y_test)
    accuracy = results[1]/results[0]
    print('Weighted hard voting accuracy: ', accuracy)
    return accuracy

In [104]:
#generates the predictions in a format to be fed to the metalearner either for training where mtype="blending" or to test the metalearner where mtype="full"
#since in blending, models are later refit on the full data

def get_prediction_set_crisp(models, y_test, y_holdout, mtype):
  preds = join_model_predictions_crisp(models, y_test, y_holdout, mtype)
  labels = []
  for m in preds:
    class_labels = np.argmax(m, axis=1)
    class_labels = class_labels.reshape(len(class_labels), 1)
    labels.append(class_labels)
  return np.hstack(labels)

In [123]:
#given a meta-learner as "blender", builds an ensemble with the models and gets the test accuracy
def blend(blender, models, y_test, y_holdout):
  meta_X = get_prediction_set_crisp(models, y_test, y_holdout, "blending")
  test_X = get_prediction_set_crisp(models, y_test, y_holdout, "full")
  blender.fit(meta_X, np.argmax(y_holdout, axis=1))
  accuracy = blender.score(test_X,np.argmax(y_test, axis=1))
  print("meta-learner: ", blender)
  print("blending test accuracy: ", accuracy)
  return accuracy

In [124]:
models = [model1, model2, model3, model4, model5, model6, model7, model10]
weighted_hard_voting(models, "full", y_test, y_holdout)
hard_voting(models, "full", y_test, y_holdout)
weighted_soft_voting(models, "full", y_test, y_holdout)
soft_voting(models, "full", y_test, y_holdout)

Weighted hard voting accuracy:  0.9494488787533257
Hard voting accuracy:  0.943747624477385
Weighted soft voting accuracy:  0.9505891296085138
Soft voting accuracy:  0.9483086278981376


0.9483086278981376

In [151]:
models=[model1, model2, model3, model4, model5, model6, model7, model8, model9, model10]

In [152]:
blender = LogisticRegression(solver ='lbfgs',multi_class ='multinomial',max_iter = 300)
blend(blender, models, y_test, y_holdout)

meta-learner:  LogisticRegression(max_iter=300, multi_class='multinomial')
blending test accuracy:  0.9137210186240973


0.9137210186240973

In [153]:
lrmodel = LogisticRegression(solver ='lbfgs',multi_class ='multinomial',max_iter = 200)
svckernel =  SVC(gamma ='auto', probability = True)
treemodel =  DecisionTreeClassifier()
knn = KNeighborsClassifier(7)
svclinear =  SVC(kernel="linear", C=0.025, probability=True)
gpc = GaussianProcessClassifier(1.0 * RBF(1.0))
rf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
mlpc = MLPClassifier(alpha=1, max_iter=1000)
ada = AdaBoostClassifier()
gauss =GaussianNB()
quad = QuadraticDiscriminantAnalysis()

blenders = [lrmodel, svckernel, treemodel, knn, svclinear, rf, mlpc, ada, gauss, quad]

In [150]:
for b in blenders:
  blend(b, models, y_test, y_holdout)
  print("_________________")

meta-learner:  LogisticRegression(max_iter=200, multi_class='multinomial')
blending test accuracy:  0.9160015203344736
_________________
meta-learner:  SVC(gamma='auto', probability=True)
blending test accuracy:  0.9289243633599392
_________________
meta-learner:  DecisionTreeClassifier()
blending test accuracy:  0.9232231090839985
_________________
meta-learner:  KNeighborsClassifier(n_neighbors=7)
blending test accuracy:  0.9266438616495629
_________________
meta-learner:  SVC(C=0.025, kernel='linear', probability=True)
blending test accuracy:  0.9080197643481566
_________________
meta-learner:  RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10)
blending test accuracy:  0.9243633599391866
_________________
meta-learner:  MLPClassifier(alpha=1, max_iter=1000)
blending test accuracy:  0.9315849486887116
_________________
meta-learner:  AdaBoostClassifier()
blending test accuracy:  0.8608893956670467
_________________
meta-learner:  GaussianNB()
blending test accuracy:

In [121]:
blend(blender, models, y_test, y_holdout)

0.9258836944127709

## GA

In [ ]:
#same function, different parameter

#1) soft_voting
#2) bagging_soft_voting
#3) weighted_soft_voting
#4) bagging_weighted_soft_voting

#5) hard_voting
#6) bagging_hard_voting
#7) weighted_hard_voting
#8) bagging_weighted_hard_voting

#blending_LR
#blending_SVCK
#blending_tree
#blending_knn
#blending_SVCL
#blending_GPC
#blending_RF
#blending_MLPC
#blending_ADA
#blending_gauss
#blending_quad

#metalearners for blending
#lrmodel = ('LR',LogisticRegression(solver ='lbfgs',multi_class ='multinomial',max_iter = 200))
#svckernel = ('SVC', SVC(gamma ='auto', probability = True))
#treemodel = ('DTC', DecisionTreeClassifier())
#knn = ('KNN', KNeighborsClassifier(7))
#svclinear = ('SVCL', SVC(kernel="linear", C=0.025, probability=True))
#gpc = ('GPC',GaussianProcessClassifier(1.0 * RBF(1.0)))
#rf = ('RF',RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1))
#mlpc = ('MLPC',MLPClassifier(alpha=1, max_iter=1000))
#ada = ('ADA',AdaBoostClassifier())
#gauss = ('GAUSS',GaussianNB())
#quad = ('QUAD',QuadraticDiscriminantAnalysis())

### GA helper functions

In [ ]:
#chromosome_architecture defines the structure of the chromosome given the list of models and mechanisms 
#start and end indices
#adaptable to any number of mechanisms

def chromosome_architecture(models, mechanisms):
  #since models are binary encoded depending on their index, the length of the model portion of the chromosome is equal to the number of potential models
  model_bits = len(models)

  #mechanism bits will be the bits needed to represent x mechanisms
  #since there can only be 1 mechanism chosen per ensemble
  #so we do one-hot encoding

  mechanism_bits = len(mechanisms).bit_length()

  #dictionaries that can get mechanism from its number and vice versa
  mechanism_to_int = dict((m, i) for i, m in enumerate(mechanisms))
  int_to_mechanism = dict((i, m) for i, m in enumerate(mechanisms))

  #total chromosome length
  total_chromosome_length = model_bits+ mechanism_bits

  #prints and returns the specified parameters
  print(model_bits, mechanism_bits, mechanism_to_int, int_to_mechanism, total_chromosome_length)
  return model_bits, mechanism_bits, mechanism_to_int, int_to_mechanism, total_chromosome_length



#given a chromosome (list of binary numbers), and chromosome architectur, decode into ensemble parameters
#returns a tuple wher [0] = list of models used and [1] = ensemble mechanism

def chromosome_decoder(chromosome, chromosome_architecture):

  model_bits = chromosome_architecture[0]
  mechanism_bits = chromosome_architecture[1]
  mechanism_to_int = chromosome_architecture[2]
  int_to_mechanism = chromosome_architecture[3]

  model_portion = chromosome[0:model_bits]

  mechanism_start = model_bits
  mechanism_end = model_bits+mechanism_bits
  mechanism_portion = chromosome[mechanism_start:mechanism_end]
  print('model encoding: ', model_portion)
  print('mechanism encoding: ', mechanism_portion)

  print("MODELS: ")
  models_used = []
  for idx, i in enumerate(model_portion):
    if i==1:
      print(models[idx])
      models_used.append(models[idx])
  
  print("Mechanism: ")
  #get the mechanism as a binary string
  mechanism_code = ''.join([str(x) for x in mechanism_portion])

  #have the program raise an exception for illegal mechanism coding
  #example: we have 10 mechanisms --> 9 binary numbers (+0) so 10 possibilities
  #4 bits needed to cover 10 possibilities
  #last legal binary number is 1001
  #1011, 1101, 1111... etc all illegal

  try:
    #transform binary mechanism code to int, and check in dictionary
    mechanism_name = int_to_mechanism[int(mechanism_code, 2)]
  except:
    #if not found, then the individual is illegal
      raise Exception("Mechanism out of bounds")
      return -1
  else:
    print(mechanism_name)
    return models_used, mechanism_name

### GA Set-Up

In [ ]:
#VARIABLES

#ALREADY TRAINED MODELS

models = [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10]

mechanisms = ['soft_voting', 'hard_voting']

In [ ]:
#NON VARIABLE

arch = chromosome_architecture(models, mechanisms)

def fitness_func(solution, solution_idx):
  print("--------------")
  print('solution: ', solution)
  try:
    #if the mechanism chosen is out of bounds (not valid) --> fitness = -1
    #or if all model portion bits are 0 --> none selected
    #the chromosome_decoder will throw an exception
    #which will be caught here and result in a fitness = -1
    parameters = chromosome_decoder(solution, arch)
  except:
    print('illegal individual - mechanism out of bounds')
    return -1 #negative fitness, so it is discounted for future generations

  if len(parameters[0])==0:
    print('illegal individual - no model selected')
    return -1

  models = parameters[0]
  mechanism = parameters[1]

  # Ensemble of Models 
  estimator = models

  #NEED TO DEFINE DATA BEFORE RUNNING THIS

  #print('ENSEMBLE')
  if mechanism == 'hard_voting':
    hard_voting = VotingClassifier(estimators = estimator, voting ='hard') 
    hard_voting.fit(X_train, y_train) 
    y_pred = hard_voting.predict(X_test)
    #print(y_pred)
    #print(y_test)
    hardscore = accuracy_score(y_test, y_pred) 
    #print("Hard Voting Score", hardscore) 
    print('fitness: ', hardscore)
    return hardscore

  if mechanism == 'soft_voting':

    softvoting = VotingClassifier(estimators = estimator, voting ='soft') 
    softvoting.fit(X_train, y_train) 
    y_pred = softvoting.predict(X_test)
    #print(y_pred)
    #print(y_test)
    softscore = accuracy_score(y_test, y_pred) 
    #print("Soft Voting Score ", score2) 
    print('fitness: ', softscore)
    return softscore
